### Confidence Intervals for the ERP (Method 1) <a id="cis-m1"></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib osx

In [2]:
from scipy.io import loadmat       # Import function to read data.
data = loadmat('EEG-1.mat')
EEGa = data['EEGa']
EEGb = data['EEGb']
t = data['t'][0]
ntrials = EEGa.shape[0]
nsamples = EEGa.shape[1]

In [ ]:
mn = EEGa.mean(0)  # Compute the mean across trials (the ERP)

In [ ]:
plt.plot(t,mn)

In [ ]:
sd = EEGa.std(0)  # Compute the std across trials

In [ ]:
sdmn = sd / sqrt(ntrials)  # Compute the std of the mean

In [ ]:
plt.plot(t, mn, 'k', lw=3)              # Plot the ERP of condition A
plt.plot(t, mn + 2 * sdmn, 'k:', lw=1)  # ... and include the upper CI
plt.plot(t, mn - 2 * sdmn, 'k:', lw=1)  # ... and the lower CI
plt.xlabel('Time [s]')                     # Label the axes
plt.ylabel('Voltage [$\mu$ V]')
plt.title('ERP of condition A')            # ... provide a useful title
plt.hlines(0, t[0], t[-1]);

### Confidence Intervals for the ERP (Method 2)

In [ ]:
# Example: Draw 1000 integers with replacement from [0, 1000)
i = np.random.randint(0, ntrials, size=ntrials)
print(i)

In [ ]:
EEG0 = EEGa[i,:]        # Create a resampled EEG.
                        # Q: What is the shape of the resampled EEG?

In [ ]:
ERP0 = EEG0.mean(0)     # Create the resampled ERP
                        # Q: What is the difference between the resampled EEG and resampled ERP?
                        # Q: What does the resampled ERP look like?

In [ ]:
plt.plot(t,ERP0)

In [ ]:
N_resample = 3000
ERP0 = np.zeros((N_resample, nsamples))
for k in arange(0,N_resample):
    i = np.random.randint(0, ntrials, size=ntrials);  # Create resampled indices.
    EEG0 = EEGa[i,:]                                  # Create a resampled EEG.
    ERP0[k,:] = EEG0.mean(0)                          # Create the resampled ERP

In [ ]:
# Plot some examples.
plt.plot(ERP0[0,:]);
plt.plot(ERP0[1,:]);

In [ ]:
ERP0_sorted = np.sort(ERP0,0)             # Sort each column of the resampled ERP
ciL = ERP0_sorted[int(0.025*N_resample)]  # Determine the lower CI
ciU = ERP0_sorted[int(0.975*N_resample)]  # ... and the upper CI
mnA = EEGa.mean(0)                        # Determine the ERP for condition A
plot(t, mnA, 'k', lw=3)                   # ... and plot it
plot(t, ciL, 'k:')                        # ... and plot the lower CI
plot(t, ciU, 'k:')                        # ... and the upper CI
hlines(0, 0, 1, 'b')                      # plot a horizontal line at 0
plt.xlabel('Time [s]')                    # ... and label the axes
plt.ylabel('Voltage');

### Comparing ERPs <a class="anchor" id="comparing-erps"></a>

In [5]:
erpA = EEGa.mean(0)         # Compute the mean across trials (the ERP)
sdA  = EEGa.std(0)          # Compute the sd across trials 
sdmnA= sdA / sqrt(ntrials)  # Compute the s.d.m.n

erpB = EEGb.mean(0)         # Compute the mean across trials (the ERP)
sdB  = EEGb.std(0)          # Compute the sd across trials 
sdmnB= sdB / sqrt(ntrials)  # Compute the s.d.m.n

plt.plot(t, erpA, 'r', lw=3, label='condition A')  # Plot the ERP of condition A
plt.plot(t, erpA + 2 * sdmnA, 'r:', lw=1)           # ... and include the upper CI
plt.plot(t, erpA - 2 * sdmnA, 'r:', lw=1)           # ... and the lower CI

plt.plot(t, erpB, 'b', lw=3, label='condition B')  # Plot the ERP of condition B
plt.plot(t, erpB + 2 * sdmnB, 'b:', lw=1)           # ... and include the upper CI
plt.plot(t, erpB - 2 * sdmnB, 'b:', lw=1)           # ... and the lower CI

plt.title('ERP of conditions A and B')
legend()

### A Bootstrap Test to Compare ERPs <a id="bootstrap"></a>

In [6]:
mnA = np.mean(EEGa,0)          # Determine ERP for condition A
mnB = np.mean(EEGb,0)          # Determine ERP for condition B
mnD = mnA - mnB                # Compute the differenced ERP
stat = np.max(np.abs(mnD))     # Compute the statistic
print('stat = {:.4f}'.format(stat))

stat = 0.2884


In [7]:
EEG = np.vstack((EEGa, EEGb))  # Step 1. Merge EEG data from all trials
np.random.seed(123)            # Fix seed for reproducibility

N_resample = 3000;
stat0 = np.zeros(N_resample)
for k in arange(0,N_resample):
    
    i = np.random.randint(0, 2*ntrials, size=ntrials);    # Create resampled indices.
    EEG0 = EEG[i,:]                                       # Create a resampled EEG.
    ERP0_A = EEG0.mean(0)                                 # Create a resampled ERP.
    
    i = np.random.randint(0, 2*ntrials, size=ntrials);    # Create resampled indices.
    EEG0 = EEG[i,:]                                       # Create a resampled EEG.
    ERP0_B = EEG0.mean(0)                                 # Create a resampled ERP.
    
    stat0[k] = np.max(np.abs(ERP0_A - ERP0_B))

In [9]:
plt.figure()
plt.hist(stat0, bins='auto')
plt.vlines(stat, 0, 100)
np.size(np.where(stat0>stat)) / stat0.size

0.006